# **Cluster de objetos espaciales: K-medias**

En este ejemplo vamos a hacer un ejemplo de agrupación con datos espaciales de patrones de puntos. S

En esta práctica voy a utilizar los datos de recogidas de [UBER en la ciudad de Nueva York que están disponibles en Kagle](https://www.kaggle.com/datasets/tekbahadurkshetri/uber-clustering). Para vuestra comodidad os he dejado los datos en el archivo "uber_clean.csv".

El Objetivo es hacer un cluster de posiciones

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

#conda install -c conda-forge folium
import folium



In [ ]:
df = pd.read_csv('datos/uber_clean.csv')  
print(df.shape)
print(df.info())
df.head()




Para que dure algo menos de tiempo la práctica voy a hacer una subselección de sólo 50.000 registros


In [ ]:
df=df.sample(n=5000, frac=None, replace=False, weights=None, random_state=123456)
df.shape

# Selección de variables para el Kmeans
Selecciono la longitud y la latitud para hacer el cluster

In [ ]:
x = df[["Lat", "Lon"]]
x.head()

# Agrupación inicial

In [ ]:
model = KMeans(n_clusters=3)
y_kmeans = model.fit_predict(x)
type(y_kmeans)

Hago la representación

In [ ]:
plt.scatter(df['Lon'], df['Lat'], c=y_kmeans)

Puedo probar con varios numeros de grupos visulamente

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 4))

# Resultados para K = 2
# ==============================================================================
y_kmeans= KMeans(n_clusters=2, n_init=25, random_state=123).fit_predict(X=x)
ax[0].scatter(
        x = df['Lon'],
        y = df['Lat'],
        c = y_kmeans,
        marker    = 'o',
        edgecolor = 'black'
    )
ax[0].set_title('KMeans K=2');

# Resultados para K = 6
# ==============================================================================
y_kmeans = KMeans(n_clusters=6, n_init=25, random_state=123).fit_predict(X=x)
ax[1].scatter(
        x = df['Lon'],
        y = df['Lat'],
        c = y_kmeans,
        marker    = 'o',
        edgecolor = 'black'
    )
ax[1].set_title('KMeans K=6');

## Selección óptima de clústeres

In [ ]:
# Método elbow para identificar el número óptimo de clusters
# (identificar aquel punto de la curva (codo) a partir del cual la mejora deja de ser notable)
# ==============================================================================
range_n_clusters = range(1, 15)
inertias = []

for n_clusters in range_n_clusters:
    modelo_kmeans = KMeans(
                        n_clusters   = n_clusters,
                        n_init       = 20,
                        random_state = 123
                    )
    modelo_kmeans.fit(x)
    inertias.append(modelo_kmeans.inertia_)

fig, ax = plt.subplots(1, 1, figsize=(6, 3.84))
ax.plot(range_n_clusters, inertias, marker='o')
ax.set_title("Evolución de la varianza intra-cluster total")
ax.set_xlabel('Número clusters')
ax.set_ylabel('Intra-cluster (inertia)');

In [ ]:
# Método silhouette para identificar el número óptimo de clusters
# (número óptimo de clusters aquel que maximiza la media del *silhouette coeficient* de todas las observaciones)
# ==============================================================================
range_n_clusters = range(2, 8)
valores_medios_silhouette = []

for n_clusters in range_n_clusters:
    modelo_kmeans = KMeans(
                        n_clusters   = n_clusters,
                        n_init       = 20,
                        random_state = 123
                    )
    cluster_labels = modelo_kmeans.fit_predict(x)
    silhouette_avg = silhouette_score(x, cluster_labels)
    valores_medios_silhouette.append(silhouette_avg)

fig, ax = plt.subplots(1, 1, figsize=(6, 3.84))
ax.plot(range_n_clusters, valores_medios_silhouette, marker='o')
ax.set_title("Evolución de media de los índices silhouette")
ax.set_xlabel('Número clusters')
ax.set_ylabel('Media índices silhouette');

# Agrupación final
**Una vez seleccionado el numero de Clusters óptimo guardo la solución**

In [ ]:
model = KMeans(
               n_clusters = 6,
               n_init       = 20,
               random_state = 123
               )
y_kmeans = model.fit_predict(x)
type(y_kmeans)

Guardo el resultado como una columna adicional

In [ ]:
df['y'] = y_kmeans
df.head(n=10)

In [ ]:
df.y.value_counts()

In [ ]:
plt.scatter(df['Lon'], df['Lat'], c=df['y'])

# Representación de los puntos en el plano

Primero seleccione el mapa sobre el que voy a representar los puntos

In [ ]:
map = folium.Map(location=[40.7128, -74.0060], zoom_start=10,tiles = "openstreetmap")
map

## Ahora dibujamos cada punto en el mapa anterior.


La otra opción es seleccionando directamente del data frame

In [28]:
# for i in range (0,len(df)):  # No voy a representar todos porque son muchos, me quedo con los 1000 primeros
for i in range (0,1000):
  # primero defino los colores y los textos   
  if df.iloc[i]['y'] == 0:
       color_f="cornflowerblue"
       texto_pop="Pertenezco al cluster 1"
  elif df.iloc[i]['y'] == 1:
       color_f="darkred"
       texto_pop="Pertenezco al cluster 2"
  elif df.iloc[i]['y'] == 2:
       color_f="darkolivegreen"
       texto_pop="Pertenezco al cluster 3"
  elif df.iloc[i]['y'] == 3:
       color_f="chocolate"
       texto_pop="Pertenezco al cluster 4"
  elif df.iloc[i]['y'] == 4:
       color_f="indigo"
       texto_pop="Pertenezco al cluster 5"    
  else:
       color_f="gold"
       texto_pop="Pertenezco al cluster 6"
        
  
        
  folium.CircleMarker([df.iloc[i]['Lat'], df.iloc[i]['Lon']], radius=15, color=color_f, fill=True, fill_opacity=0.6, tooltip=texto_pop, popup="cluster {}".format(df.iloc[i]['y']+1),).add_to(map)
    



In [ ]:
map

In [22]:
map.save('mapaUBER.html')